In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from utils import *
from keras.optimizers import Adam

image_size = (768, 768) #(512,512) (720, 1280)
bs = 4

better_model = False
load_pretrained_weights = False

losses = sparse_crossentropy_ignoring_last_label
metrics = {'pred_mask' : [Jaccard, sparse_accuracy_ignoring_last_label]}

backbone = 'mobilenetv2' #mobilenetv2, xception

NET = 'deeplab_' + backbone
PATH = '/mnt/deep_learning/'

n_classes = 2


Using TensorFlow backend.


In [2]:
SegClass = SegModel(PATH, image_size)
SegClass.set_batch_size(bs)


model = SegClass.create_seg_model(net='original',n=n_classes, load_weights=load_pretrained_weights,
                                      multi_gpu=False, backbone=backbone)

model.compile(optimizer = Adam(lr=7e-4, epsilon=1e-8, decay=1e-6), sample_weight_mode = "temporal",
              loss = losses, metrics = metrics)
print('Weights path:', SegClass.modelpath)















Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use `tf.cast` instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `tf.cast` instead.
Weights path: /mnt/deep_learning//weights/mobilenetv2_original.h5


In [3]:
monitor = 'Jaccard'
mode = 'max'

train_generator = SegClass.create_generators(blur=5,crop_shape=None, mode='train', n_classes=n_classes,
                                             horizontal_flip=True, vertical_flip=False, brightness=0.3,
                                             rotation=False, zoom=0.1, validation_split=.15, seed=7, do_ahisteq=False)
valid_generator = SegClass.create_generators(blur=0, crop_shape=None, mode='validation',
                                             n_classes=n_classes, horizontal_flip=True, vertical_flip=False,
                                             brightness=.1, rotation=False, zoom=.05, validation_split=.15,
                                             seed=7, do_ahisteq=False)


In [4]:
def build_callbacks(tf_board = False):
    tensorboard = TensorBoard(log_dir='./logs/'+SegClass.net, histogram_freq=0,
                        write_graph=False, write_images = False)
    checkpointer = ModelCheckpoint(filepath = SegClass.modelpath, verbose=1, save_best_only=True, save_weights_only=True,
                                    monitor = 'val_{}'.format(monitor), mode = mode)
    stop_train = EarlyStopping(monitor = 'val_{}'.format(monitor), patience=100, verbose=1, mode = mode)
    reduce_lr = ReduceLROnPlateau(monitor = 'val_{}'.format(monitor), factor=0.5,
                patience=5, min_lr=1e-6)
    if tf_board:
        callbacks = [reduce_lr, stop_train, tensorboard]
    else:
        callbacks = [checkpointer, reduce_lr, stop_train]
    return callbacks


In [5]:
import sys
oldStdout = sys.stdout
file = open('/mnt/deep_learning/logs/log.out', 'w')
sys.stdout = file

In [ ]:
callbacks = build_callbacks(tf_board = False)
SegClass.set_num_epochs(10)
history = SegClass.train_generator(model, train_generator, valid_generator, callbacks, mp = False)

# Load original deeplab and subpixel-deeplab
# model1 = SegClass.create_seg_model(net='original',n=n_classes, load_weights=False, multi_gpu=False, backbone=backbone)


In [ ]:
sys.stdout = oldStdout

In [ ]:
# Testing a few images

In [ ]:
print("done now")

In [ ]:
test_img1 = cv2.imread('/mnt/deep_learning/test_v2/7028c4629.jpg', 1)

In [18]:
test_img_1_p  = test_img1.reshape(1,768,768,3)

In [19]:
test_img_1_p.shape

(1, 768, 768, 3)

In [1]:
preds1 = np.argmax(model.predict(test_img_1_p), -1)[0].reshape(image_size)

NameError: name 'np' is not defined

In [ ]:
cv2.imshow("windows", preds1)